## Importamos Bibliotecas

In [4]:
# Biblioteca pandas para el manejo de datos
import pandas as pd

# EXTRACCIÓN DE LA INFORMACIÓN


In [6]:
canales = pd.read_csv('..\\data\\canales.csv', encoding='utf-8')
clientes = pd.read_csv('..\\data\\clientes.csv', encoding='utf-8')
transacciones = pd.read_csv('..\\data\\transacciones.csv', encoding='utf-8')